### Libraries

In [1]:
import cv2
import os
import numpy as np
import urllib.request

### Download model files

In [3]:
if not os.path.exists("deploy.prototxt"):
    print("Downloading deploy.prototxt...")
    urllib.request.urlretrieve(
        "https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt",
        "deploy.prototxt",
    )

if not os.path.exists("res10_300x300_ssd_iter_140000.caffemodel"):
    print("Downloading model weights...")
    urllib.request.urlretrieve(
        "https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20180205_fp16/res10_300x300_ssd_iter_140000_fp16.caffemodel",
        "res10_300x300_ssd_iter_140000.caffemodel",
    )

# Load the pre-trained Caffe model
try:
    net = cv2.dnn.readNetFromCaffe(
        "deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel"
    )
except Exception as e:
    print("Error loading model:", e)
    raise SystemExit("Failed to load face detection model")

### Methods

In [4]:
def detect_faces(img, confidence_threshold=0.5):  # Lower confidence threshold
    (h, w) = img.shape[:2]

    # Create blob with larger scale to detect smaller faces
    blob = cv2.dnn.blobFromImage(
        cv2.resize(img, (500, 500)),  # Input size
        1.0,  # Scale factor
        (500, 500),  # Spatial size
        (104.0, 177.0, 123.0),  # Mean subtraction (BGR)
        swapRB=False,  # Swap Red/Blue
        crop=False,  # Center crop
    )

    net.setInput(blob)
    detections = net.forward()

    faces = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        if confidence > confidence_threshold:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Ensure coordinates are within image bounds
            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)

            # Only add if face is reasonably sized
            face_width = endX - startX
            face_height = endY - startY
            if face_width > 20 and face_height > 20:  # Minimum face size
                faces.append([startX, startY, face_width, face_height])

    # Less aggressive NMS for crowded images
    return non_max_suppression(np.array(faces), overlapThresh=0.3)


def non_max_suppression(boxes, overlapThresh):
    # Remove duplicate detections of the same face
    # Reduce false positives in crowded scenes
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(
            idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0]))
        )

    return boxes[pick].astype("int")


def process_image(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None, 0

    # Convert to RGB (better for face detection)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = detect_faces(rgb_img)

    # Draw rectangles on original BGR image
    for x, y, w, h in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img, len(faces)

### Process images

In [7]:
# Create output directory
os.makedirs("output_images", exist_ok=True)

for filename in os.listdir("dataset"):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join("dataset", filename)
        processed_img, face_count = process_image(img_path)

        if processed_img is not None:
            output_path = os.path.join("output_images", f"detected_{filename}")
            cv2.imwrite(output_path, processed_img)
            print(f"{filename}: Detected {face_count} faces")  # Correct count

print("Processing complete!")

data (1).jpg: Detected 1 faces
data (10).jpg: Detected 4 faces
data (11).jpg: Detected 6 faces
data (12).jpg: Detected 7 faces
data (2).jpg: Detected 7 faces
data (3).jpg: Detected 15 faces
data (4).jpg: Detected 3 faces
data (5).jpg: Detected 5 faces
data (6).jpg: Detected 4 faces
data (7).jpg: Detected 1 faces
data (8).jpg: Detected 1 faces
data (9).jpg: Detected 7 faces
Processing complete!
